In [1]:
import os
import glob
import h5py
import numpy as np
import open3d as o3d
from sklearn.cluster import DBSCAN, OPTICS
import matplotlib.pyplot as plt
import plotly.graph_objects as go

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# функция для сегментации
def segment_pcd(X, pcd, points, user_eps, user_min_samp):
  clustering = DBSCAN(eps=user_eps, min_samples=user_min_samp, algorithm='ball_tree').fit(X)
  labels=clustering.labels_
  
  max_label = labels.max() + 1

  obj_points=[]
  for i in range(max_label):
    idx_labels = np.where(labels==i)
    obj_points.append(points[idx_labels])

  idx_layer=np.where(labels<0)
  colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
  colors[labels < 0] = 0
  pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

  return pcd, colors, max_label, obj_points

In [3]:
# функция для графического отображения облака точек
def draw_plot(points, colors=None):
  fig = go.Figure(
      data=[
          go.Scatter3d(
              x=points[:,0], y=points[:,1], z=points[:,2], 
              mode='markers',
              marker=dict(size=1, color=colors)
          )
      ],
      layout=dict(
          scene=dict(
              xaxis=dict(visible=False),
              yaxis=dict(visible=False),
              zaxis=dict(visible=False)
          )
      )
      )
  return fig

In [4]:
# функция присваивания цвета точке
def add_color(pcd, colors):
  if pcd.has_colors():
    colors = np.asarray(pcd.colors)
  elif pcd.has_normals():
    colors = (0.0, 0.5, 0.5) + np.asarray(pcd.normals) * 0.5
  else:
    pcd.paint_uniform_color((0.0, 0.0, 0.0))
    colors = np.asarray(pcd.colors)
  return pcd, colors

In [5]:
# функция для представления набора точек в pcd
def np_to_pcd(points):
  colors = np.zeros(points.shape)
  for i in range(0,points.shape[0]):
    colors[i][0] = 0.12156863
    colors[i][1] = 0.46666667
    colors[i][2] = 0.70588235
  point_cloud = o3d.geometry.PointCloud()
  point_cloud.points = o3d.utility.Vector3dVector(points)
  point_cloud.colors = o3d.utility.Vector3dVector(colors)
  return point_cloud

In [6]:
DATA_DIR = "/"

In [7]:
h5f = h5py.File(os.path.join(DATA_DIR, "segmentation-dataset.h5"),'r')
df = h5f.get('dataset_a')[:]
h5f.close()

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/segmentation-dataset.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)